# The Real Impact of the GIL

In `concurrent-hello-world` (thread version), we have `time.sleep()` that represents like a HTTP client request and the spinner keeps spining. That’s because a well-designed network library will release the GIL while waiting for the network.

However, with CPU intensive works, the story is different

In [1]:
import math

def is_prime(n: int) -> bool:
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    
    root = math.isqrt(n)
    for i in range(3, root + 1, 2):
        if n % i == 0:
            return False
    return True

is_prime(5_000_111_000_222_021)  # this will take about 3 secs

True

Now let's substitute `time.sleep` with `is_prime`

## Processes

In [2]:
# spinner_proc.py

import itertools
import time
from multiprocessing import Process, Event  
from multiprocessing import synchronize     


def spin(msg: str, done: synchronize.Event) -> None:  # difference to the thread version here
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')


def slow() -> int:
    is_prime(5_000_111_000_222_021)
    return 42


def supervisor() -> int:
    done = Event()
    spinner = Process(target=spin, args=('thinking!', done))  # difference to the thread version here
    print(f'spinner object: {spinner}')          
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result


def main() -> None:
    result = supervisor()
    print(f'Answer: {result}')


main()

spinner object: <Process name='Process-1' parent=58090 initial>
Answer: 42  


&#128212; The spinner is controlled by a child process, so it continues spinning normally while the primality test is computed by the parent process

## Threading

In [3]:
import itertools
import time
from threading import Thread, Event


def spin(msg: str, done: Event) -> None:  # This function will run in a separate thread. Event is used to synchronize threads 
    for char in itertools.cycle(r'\|/-'):  # This is an infinite loop because itertools.cycle yields one character at a time
        status = f'\r{char} {msg}'  # Trick for text-mode animation: move the cursor back to the start of the line with the carriage return ASCII control character ('\r').
        print(status, end='', flush=True)
        if done.wait(.1):  # .wait returns True when the event is set() by another thread; if the timeout elapses, it returns False
            break  # Exit the infinite loop
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')  # Clear the status line by overwriting with spaces and moving the cursor back to the beginning.


def slow() -> int:
    is_prime(5_000_111_000_222_021)
    return 42


def supervisor() -> int:  # supervisor will return the result of slow
    done = Event()  # Event is used to coordinate the main thread and spinner thread
    spinner = Thread(target=spin, args=('thinking!', done))  # Run a new thread with the spin function as target
    print(f'spinner object: {spinner}')  # the spinner object will be <Thread(Thread-1, initial)>, where initial means the thread is not started yet
    spinner.start()  # Start the spinner thread.
    result = slow()  # Call slow, which blocks the main thread. Meanwhile, the spinner thread is running.
    done.set()  # Set the Event flag to True; this will terminate the for loop inside the spin function
    spinner.join()  # Wait until the spinner thread finishes.
    return result


def main() -> None:
    result = supervisor()  # Run the supervisor function.
    print(f'Answer: {result}')


main()

spinner object: <Thread(Thread-4 (spin), initial)>
Answer: 42  


&#128212; The threaded version of the experiment worked—despite the GIL—because Python periodically interrupts threads (Python suspends the running thread every 5ms). Our example used only two threads: One doing compute-intensive work, and the other driving the animation only 10 times per second

## Coroutines

In [6]:
!python spinner_async_cpu_intensive.py

spinner object: <Task pending name='Task-2' coro=<spin() running at /home/dk/Desktop/projects/programming/fluent-python/19-concurrency/spinner_async_cpu_intensive.py:7>>
5000111000222021 is a prime: True


The spin did not appear at all because the flow of control will pass from `supervisor` to `is_prime`. When `is_prime` returns, `supervisor` resumes and cancel the spinner task before it is executed even once.